# Feature-Aware Data Augmentation

Augmentation policies to consider:
- Simple policy randomly sampling of n transforms 
- Constrained sampling policy with a blacklist of transforms to avoid 
- Feature-aware augmentation policy where transforms are picked based on their (transform, feature) behavior

In [2]:
# !pip install codecarbon

In [32]:
!codecarbon init


Welcome to CodeCarbon, here is your experiment id:
87abc9a8-d71c-4913-a654-80a754736084 (from ./.codecarbon.config)



In [2]:
# data
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from datasets import load_dataset, load_from_disk
from datasets.utils.logging import disable_progress_bar

# amrs
import amrlib
import penman

# transform
import sibyl
import time
import torch
import inspect
import random
from functools import partial

# eval pipeline
import pandas as pd
from transformers import pipeline
from huggingface_hub import HfApi, ModelFilter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from scipy.special import softmax

# train pipeline
import shutil
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    Trainer, 
    TrainingArguments, 
    EarlyStoppingCallback
)

# cleanlab pipeline
from cleanlab.filter import find_label_issues

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

from codecarbon import EmissionsTracker

In [3]:
from nlaugmenter.transformations.emojify                                   import EmojifyTransformation
from nlaugmenter.transformations.synonym_insertion                         import SynonymInsertion
from nlaugmenter.transformations.back_translation                          import BackTranslation
from nlaugmenter.transformations.noun_compound_paraphraser                 import NounCompoundParaphraser
from nlaugmenter.transformations.diacritic_removal                         import DiacriticRemoval
from nlaugmenter.transformations.azerty_qwerty_chars_swap                  import AzertyQwertyCharsSwap
from nlaugmenter.transformations.add_hashtags                              import HashtagGeneration
from nlaugmenter.transformations.replace_spelling                          import SpellingTransformation
from nlaugmenter.transformations.token_replacement                         import TokenReplacement
from nlaugmenter.transformations.auxiliary_negation_removal                import SentenceAuxiliaryNegationRemoval
from nlaugmenter.transformations.use_acronyms                              import UseAcronyms
from nlaugmenter.transformations.yoda_transform                            import YodaPerturbation
from nlaugmenter.transformations.factive_verb_transformation               import FactiveVerbTransformation
from nlaugmenter.transformations.protaugment_diverse_paraphrase            import ProtaugmentDiverseParaphrase
from nlaugmenter.transformations.replace_financial_amounts                 import ReplaceFinancialAmount
from nlaugmenter.transformations.americanize_britishize_english            import AmericanizeBritishizeEnglish
from nlaugmenter.transformations.punctuation                               import PunctuationWithRules
from nlaugmenter.transformations.urban_dict_swap                           import UrbanThesaurusSwap
from nlaugmenter.transformations.filler_word_augmentation                  import FillerWordAugmentation
from nlaugmenter.transformations.synonym_substitution                      import SynonymSubstitution
from nlaugmenter.transformations.style_paraphraser                         import StyleTransferParaphraser
from nlaugmenter.transformations.random_upper_transformation               import RandomUpperPerturbation
from nlaugmenter.transformations.weekday_month_abbreviation                import WeekdayMonthAbbreviation
from nlaugmenter.transformations.city_names_transformation                 import CityNamesTransformation
from nlaugmenter.transformations.sentence_additions                        import SentenceAdditions
from nlaugmenter.transformations.change_date_format                        import ChangeDateFormat
from nlaugmenter.transformations.tense                                     import TenseTransformation
from nlaugmenter.transformations.dyslexia_words_swap                       import DyslexiaWordsSwap
from nlaugmenter.transformations.gender_culture_diverse_name               import GenderCultureDiverseName
from nlaugmenter.transformations.gender_neutral_rewrite                    import GenderNeutralRewrite
from nlaugmenter.transformations.lost_in_translation                       import LostInTranslation
from nlaugmenter.transformations.insert_abbreviation                       import AbbreviationInsertionEN
from nlaugmenter.transformations.random_deletion                           import RandomDeletion
from nlaugmenter.transformations.transformer_fill                          import TransformerFill
from nlaugmenter.transformations.concat_monolingual                        import ConcatMonolingual
from nlaugmenter.transformations.country_state_abbreviation_transformation import CountryStateAbbreviation
from nlaugmenter.transformations.butter_fingers_perturbation               import ButterFingersPerturbation
from nlaugmenter.transformations.replace_abbreviation_and_acronyms         import ReplaceAbbreviations
from nlaugmenter.transformations.slangificator                             import Slangificator
from nlaugmenter.transformations.yes_no_question                           import YesNoQuestionPerturbation
from nlaugmenter.transformations.gender_swap                               import GenderSwap
from nlaugmenter.transformations.close_homophones_swap                     import CloseHomophonesSwap
from nlaugmenter.transformations.simple_ciphers                            import SimpleCiphers
from nlaugmenter.transformations.change_person_named_entities              import ChangePersonNamedEntities
from nlaugmenter.transformations.greetings_and_farewells                   import GreetingsAndFarewells
from nlaugmenter.transformations.discourse_marker_substitution             import DiscourseMarkerSubstitution
from nlaugmenter.transformations.summarization_transformation              import Summarization
from nlaugmenter.transformations.visual_attack_letters                     import VisualAttackLetters
# from nlaugmenter.transformations.sentence_reordering                       import SentenceReordering
from nlaugmenter.transformations.change_char_case                          import ChangeCharCase
from nlaugmenter.transformations.antonyms_substitute                       import AntonymsSubstitute
from nlaugmenter.transformations.mix_transliteration                       import MixTransliteration
from nlaugmenter.transformations.disability_transformation                 import DifferentAbilityTransformation
from nlaugmenter.transformations.abbreviation_transformation               import Abbreviate
from nlaugmenter.transformations.replace_with_hyponyms_hypernyms           import ReplaceHypernyms
from nlaugmenter.transformations.replace_with_hyponyms_hypernyms           import ReplaceHyponyms
from nlaugmenter.transformations.grapheme_to_phoneme_transformation        import PhonemeSubstitution
from nlaugmenter.transformations.multilingual_lexicon_perturbation         import MultilingualLexiconPerturbation
from nlaugmenter.transformations.multilingual_back_translation             import MultilingualBackTranslation
from nlaugmenter.transformations.unit_converter                            import UnitConverter
from nlaugmenter.transformations.adjectives_antonyms_switch                import SentenceAdjectivesAntonymsSwitch
from nlaugmenter.transformations.correct_common_misspellings               import CorrectCommonMisspellings
from nlaugmenter.transformations.neural_question_paraphraser               import NeuralParaphaserPerturbation
from nlaugmenter.transformations.subject_object_switch                     import SentenceSubjectObjectSwitch
from nlaugmenter.transformations.numeric_to_word                           import NumericToWord
from nlaugmenter.transformations.multilingual_dictionary_based_code_switch import MultilingualDictionaryBasedCodeSwitch
from nlaugmenter.transformations.diverse_paraphrase                        import DiverseParaphrase
from nlaugmenter.transformations.english_inflectional_variation            import EnglishInflectionalVariation
from nlaugmenter.transformations.replace_numerical_values                  import ReplaceNumericalValues
from nlaugmenter.transformations.speech_disfluency_perturbation            import SpeechDisfluencyPerturbation
from nlaugmenter.transformations.whitespace_perturbation                   import WhitespacePerturbation
from nlaugmenter.transformations.contraction_expansions                    import ContractionExpansions
from nlaugmenter.transformations.color_transformation                      import ColorTransformation
from nlaugmenter.transformations.contextual_meaning_perturbation           import ContextualMeaningPerturbation
from nlaugmenter.transformations.pig_latin                                 import PigLatin
from nlaugmenter.transformations.leet_letters                              import LeetLetters
from nlaugmenter.transformations.hashtagify                                import HashtagifyTransformation
# from nlaugmenter.transformations.propbank_srl_roles                        import CheckSrl
from nlaugmenter.transformations.geo_names_transformation                  import GeoNamesTransformation

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
torch.use_deterministic_algorithms(False)
disable_progress_bar()

In [6]:
# helper functions

def normalize_minmax(df):
    for column in df.columns:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())  
    return df

def normalize_sum(df):
    for column in df.columns:
        df[column] = df[column] / df[column].sum()
    return df

def augment_data(batch, transform, keep_originals=True):
    new_texts, new_labels = [], []
    for text, label in zip(batch['text'], batch['label']):
        new_text, new_label = transform.apply([text], [label])
        new_texts.extend(new_text)
        new_labels.extend(new_label)
    if keep_originals:
        return {"text": batch['text'] + new_texts, "label": batch['label'] + new_labels}
    else:
        return {"text": new_texts, "label": new_labels}
    
def percent_dataset_changed(d1, d2):
    return sum([t1['text'] != t2['text'] for t1, t2 in zip(d1, d2)]) / len(d1) 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    acc = accuracy_score(labels, predictions.argmax(-1))
    precision, recall, fbeta_score, support = precision_recall_fscore_support(
        y_true=labels, 
        y_pred=predictions.argmax(-1), 
        average="weighted", 
        zero_division=0)
    return { 'accuracy': acc , 
             'precision': precision, 
             'recall': recall, 
             'fbeta_score': fbeta_score} 

def compute_accuracy(predictions, labels):
    if not isinstance(labels, np.ndarray):
        labels = np.array(labels)
    if len(labels.shape) > 1:
        acc = acc_at_k(labels, predictions, k=2)       
    else:
        acc = accuracy_score(labels, np.argmax(predictions, -1))
    return acc

def vectorize(output):
    sorted_output = sorted(output, key=lambda d: d['label']) 
    probs = torch.tensor([d['score'] for d in sorted_output])
    return probs

def sample_transforms(transforms, p, n=2, replace=False):
    return np.random.choice(transforms, size=n, p=p, replace=replace).tolist()

def transforms_to_ids(sampled_transforms, all_transforms):
    transforms_ids = [all_transforms.index(i) for i in sampled_transforms]
    transforms_applied = np.zeros(len(all_transforms), dtype=np.int32)
    transforms_applied[transforms_ids] = 1
    return transforms_applied

def policy_heatmap(policy, transforms, featurizers):
    t_names = [t.transform_class.__name__ for t in transforms]
    f_names = [f.__name__ for f in featurizers]
    df = pd.DataFrame(policy)
    df.columns = f_names
    df.index = t_names
    sns.heatmap(df)
    plt.show()
    
def implement_policy_probabilities(policy, features):
    default_probability = policy.mean(axis=1)
    policy_probs = []
    for f in features:
        available_features = np.nonzero(f)[0]
        if len(available_features) == 0:
            probs = default_probability
        else:
            probs = policy[:, available_features].mean(axis=1)
        policy_probs.append(probs)
    return np.array(policy_probs)

In [7]:
class Transform:
    def __init__(self, transform_class, num_outputs=1, task_name="sentiment"):
        self.transform_class = transform_class
        self.num_outputs = num_outputs
        self.task_name = task_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.intakes_target = False
        self.is_batched = False
        
        # setting class attributes
        if 'to_tense' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with to_tense='past'") # future & random don't work
            self.transform_instance = self.transform_class(to_tense="past")
        elif 'source_lang' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with source_lang='es'") 
            self.transform_instance = self.transform_class(source_lang="es")
        elif 'task_name' in inspect.signature(self.transform_class).parameters:
            print(f"initializing {self.transform_class.__name__} with task_name='{task_name}', return_metadata=True") 
            self.transform_instance = self.transform_class(task_name=self.task_name, return_metadata=True)
        elif isinstance(self.transform_class, LostInTranslation):
            print(f"initializing {self.transform_class.__name__} with device=0")
            self.transform_instance = self.transform_class(device=0)
        else:
            print(f"initializing {self.transform_class.__name__}")
            self.transform_instance = self.transform_class()
        
        # setting instance attributes
        if hasattr(self.transform_instance, "max_outputs"):
            print(f"setting max_outputs={self.num_outputs}")
            self.transform_instance.max_outputs = self.num_outputs
        if hasattr(self.transform_instance, "max_paraphrases"):
            print(f"setting max_paraphrases={self.num_outputs}")
            self.transform_instance.max_paraphrases = self.num_outputs
        if hasattr(self.transform_instance, "device"):
            if self.transform_instance.device is None or self.transform_instance.device == 'cpu':
                print(f"setting device={self.device}")
                self.transform_instance.device = self.device
        
        # selecting the transformation function
        if hasattr(self.transform_class, "generate"):
            self.transform_fn = self.transform_instance.generate
        if hasattr(self.transform_class, "augment"):
            self.transform_fn = self.transform_instance.augment
        if hasattr(self.transform_class, "transform_batch"):
            self.transform_fn = self.transform_instance.transform_batch
            self.intakes_target = True
            self.is_batched = True
            
    def synced_shuffle(self, list1, list2):
        # Shuffle two lists with same order
        # Using zip() + * operator + shuffle()
        temp = list(zip(list1, list2))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        res1, res2 = list(res1), list(res2)
        return res1, res2
            
    def apply(self, texts, labels=None):
        if self.intakes_target:
            if self.is_batched:
                new_texts, new_labels = self.transform_fn((texts, labels))
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t, new_l = self.transform_fn(t, l)
                    new_texts.append(new_t)
                    new_labels.extend([new_l] * len(new_t))
        else:
            if self.is_batched:
                new_texts = self.transform_fn((texts))
                new_texts = labels
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t = self.transform_fn(t)
                    if len(new_t) > self.num_outputs:
                        new_t = new_t[:self.num_outputs]
                    new_texts.extend(new_t)
                    new_labels.extend([l] * len(new_t))
                    
        # post processing since some transformations add/remove more new outputs than expected
        if len(new_texts) == 0:
            print("no new_texts, substituting original texts...")
            new_texts = texts
        if len(new_labels) == 0:
            print("no new_labels, substituting original labels...")
            new_labels = labels
        new_texts, new_labels = self.synced_shuffle(new_texts, new_labels)
        
        expected_len = len(texts) * self.num_outputs
        new_texts = new_texts[:expected_len]
        new_labels = new_labels[:expected_len]
        
        return new_texts, new_labels

In [8]:
class AMRGraph:
    def __init__(self, amr):
        self.graph = penman.decode(amr) if not isinstance(amr, penman.graph.Graph) else amr
        self.amr_text = penman.encode(self.graph)

    def contains_concept(self, concepts):
        """
        Concepts are nodes / instances in the AMR graph.
        """
        try:
            if not isinstance(concepts, list): concepts = [concepts]
            graph_concepts = [t.target for t in self.graph.instances()]
            return any(c for c in graph_concepts if c in concepts)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

    def contains_role(self, roles):
        """
        Roles are edges in the AMR graph.
        """
        try:
            if not isinstance(roles, list): roles = [roles]
            graph_roles = [e.role for e in self.graph.edges()]
            return any(r for r in graph_roles if r in roles)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

    def contains_attribute(self, attributes):
        """
        Attributes are properties of concept nodes, i.e. relationships to 
        constant values.
        """
        try:
            if not isinstance(attributes, list): attributes = [attributes]
            graph_attrs = [a.target for a in self.graph.attributes()]
            return any(a for a in graph_attrs if a in attributes)
        except Exception as e:
            print(e)
            print(self.graph, self.amr_text)

In [9]:
# attributes =============================================================

def contains_imperative(g): return g.contains_attribute("imperative")
def contains_exlamation(g): return g.contains_attribute("expressive")
def contains_negation(g):   return g.contains_attribute("-")

# concepts ===============================================================

def contains_conjunctions(g):         return g.contains_concept(["and", "or", "contrast-01", "either", "neither"])
def contains_interrogative_clause(g): return g.contains_concept("truth-value")
def contains_question(g):             return g.contains_concept(["amr-unknown", "amr-choice"])

# roles ==================================================================

def contains_coreferences(g): return any(r for r in g.amr_text.split() if r in ['i', 'you', 'he', 'she', 'it', 'we', 'they'])
def contains_number(g):       return any(a for a in g.graph.attributes() if a.target.isnumeric())

def contains_accompanier(g):  return g.contains_role(':accompanier')
def contains_age(g):          return g.contains_role(':age')
def contains_beneficiary(g):  return g.contains_role(':beneficiary')
def contains_concession(g):   return g.contains_role(':concession')
def contains_condition(g):    return g.contains_role(':condition')
def contains_consist_of(g):   return any(r for r in g.amr_text.split() if r in [':consist-of'])
def contains_degree(g):       return g.contains_role(':degree')
def contains_destination(g):  return g.contains_role(':destination')
def contains_direction(g):    return g.contains_role(':direction')
def contains_domain(g):       return g.contains_role(':domain')
def contains_duration(g):     return g.contains_role(':duration')
def contains_example(g):      return g.contains_role(':example')
def contains_extent(g):       return g.contains_role(':extent')
def contains_frequency(g):    return g.contains_role(':frequency')
def contains_instrument(g):   return g.contains_role(':instrument')
# def contains_li(g):           return g.contains_role(':li')
def contains_location(g):     return g.contains_role(':location')
def contains_manner(g):       return g.contains_role(':manner')
def contains_medium(g):       return g.contains_role(':medium')
def contains_mod(g):          return g.contains_role(':mod')
def contains_mode(g):         return any(a for a in g.graph.attributes() if ":mode" in a.role)
def contains_name(g):         return g.contains_role(':name')
def contains_ord(g):          return g.contains_role(':ord')
def contains_part(g):         return g.contains_role(':part')
def contains_path(g):         return g.contains_role(':path')
def contains_polarity(g):     return g.contains_role(':polarity')
def contains_polite(g):       return any(r for r in g.amr_text.split() if r in [':polite'])
def contains_poss(g):         return g.contains_role(':poss')
def contains_purpose(g):      return g.contains_role(':purpose')
def contains_quant(g):        return g.contains_role(':quant')
def contains_range(g):        return g.contains_role(':range')
def contains_scale(g):        return g.contains_role(':scale')
def contains_source(g):       return g.contains_role(':source')
def contains_subevent(g):     return g.contains_role(':subevent')
def contains_time(g):         return g.contains_role(':time')
def contains_topic(g):        return g.contains_role(':topic')
def contains_unit(g):         return g.contains_role(':unit')
# def contains_value(g):        return g.contains_role(':value')
def contains_wiki(g):         return g.contains_role(':wiki')

In [10]:
class AMRFeatureExtractor:
    
    def __init__(self):
        self.featurizers = featurizers = [    
            contains_accompanier,
            contains_age,
            contains_beneficiary,
            contains_concession,
            contains_condition,
            contains_conjunctions,
            contains_consist_of,
            contains_coreferences,
            contains_degree,
            contains_destination,
            contains_direction,
            contains_domain,
            contains_duration,
            contains_example,
            contains_exlamation,
            contains_extent,
            contains_frequency,
            contains_imperative,
            contains_instrument,
            contains_interrogative_clause,
            contains_location,
            contains_manner,
            contains_medium,
            contains_mod,
            contains_mode,
            contains_name,
            contains_negation,
            contains_number,
            contains_ord,
            contains_part,
            contains_path,
            contains_polarity,
            contains_polite,
            contains_poss,
            contains_purpose,
            contains_quant,
            contains_question,
            contains_range,
            contains_scale,
            contains_source,
            contains_subevent,
            contains_time,
            contains_topic,
            contains_unit
        ]
        self.featurizers = sorted(featurizers, key=lambda f: f.__name__)
        self.amr_model   = None
        
    def load_amr_model(self, max_sent_len=128):
        self.amr_model = amrlib.load_stog_model(max_sent_len=max_sent_len)
        
    def text_to_amr(self, texts):
        if self.amr_model is None:
            self.load_amr_model()
        amr_penmans = self.amr_model.parse_sents(texts, add_metadata=False, disable_progress=True)
        amr_graphs = []
        for p in amr_penmans:
            try:
                amr_graphs.append(AMRGraph(p))
            except Exception as e: 
                print(e)
                print(p)
                amr_graphs.append(AMRGraph(p))
        return amr_graphs
    
    def generate_feature_matrix(self, graphs):
        feature_matrix = []
        for g in graphs:
            feature_vector = []
            for f in self.featurizers:
                feature_vector.append(f(g))
            feature_matrix.append(feature_vector)
        feature_matrix = np.array(feature_matrix, dtype=np.int32)
        return feature_matrix
    
    def __call__(self, texts):
        graphs = self.text_to_amr(texts)
        return self.generate_feature_matrix(graphs)
        

In [11]:
class Augmenter:
    def __init__(self, 
                 dataset,
                 transforms,
                 transform_probabilities = None, 
                 num_augmentations_per_record = 5,
                 num_transforms_to_apply = 2,
                 batch_size = 10,
                 allow_resampling = False,
                 keep_originals = False,
                 feature_extractor = None,
                 perf_extractor = None):
        
        self.dataset = dataset
        self.transforms = transforms
        self.transform_probabilities = transform_probabilities
        self.num_augmentations_per_record = num_augmentations_per_record
        self.num_transforms_to_apply = num_transforms_to_apply
        self.batch_size = batch_size
        self.allow_resampling = allow_resampling
        self.keep_originals = keep_originals
        self.feature_extractor = feature_extractor
        self.perf_extractor = perf_extractor
        
        # initializations
        self.dataset = dataset.remove_columns("idx")
        self.add_idx_to_dataset()
        self.num_transforms_available = len(self.transforms)
        
        if self.transform_probabilities is None:
            # set to uniform
            num_examples = len(self.dataset)
            num_transforms = len(self.transforms)
            uniform_policy = np.full((num_examples, num_transforms), fill_value=1/num_transforms)
            self.transform_probabilities = uniform_policy
        
    def add_idx_to_dataset(self):
        if 'idx' not in self.dataset.features:
            self.dataset = self.dataset.add_column("idx", range(len(self.dataset)))
        
    def apply_to_batch(self, batch):
        new_texts, new_labels, transforms_applied, is_changed = [], [], [], []
        for idx, text, label in zip(batch['idx'], batch['text'], batch['label']):
            actual_batch_size = len(batch['idx'])
            original_text, original_label = text, label
            for _ in range(self.num_augmentations_per_record):
                sampled_transforms = sample_transforms(self.transforms, 
                                                       p=self.transform_probabilities[idx], 
                                                       n=self.num_transforms_to_apply, 
                                                       replace=self.allow_resampling)
                transforms_applied.append(transforms_to_ids(sampled_transforms, self.transforms))
                for t in sampled_transforms:
                    try:
                        text, label = t.apply([text], [label])
                        text, label = text[0], label[0]
                    except Exception as e: 
                        print(e)
                        print(f"[Augmenter]: skipping augmentation from {t.transform_class.__name__} on text:'{text}' and label: {label}")

                # avoid adding records with empty text
                if text:
                    new_texts.append(text)
                    new_labels.append(label)
                    is_changed.append(int(original_text != text))

        if self.keep_originals:
            new_texts = batch['text'] + new_texts
            new_labels = batch['label'] + new_labels
            realized_batch_size = len(new_labels)
            transforms_applied = transforms_applied + np.zeros((actual_batch_size, len(self.transforms)), dtype=np.int32).tolist()
            is_changed = is_changed + [0] * actual_batch_size
            out = {
                "text": new_texts, 
                "label": new_labels,
                "idx": list(range(realized_batch_size)),
                "transforms_applied": [t for t in transforms_applied],
                "is_changed": is_changed
            }
        else:
            out = {
                "text": new_texts, 
                "label": new_labels, 
                "idx": list(range(len(new_labels))),
                "transforms_applied": transforms_applied,
                "is_changed": is_changed
            }

        return out
            
                                                   
    def augment(self):
        dataset = self.dataset.map(self.apply_to_batch, batched=True, batch_size=self.batch_size)
        dataset = dataset.remove_columns("idx")
    
        # feature extraction
        if self.feature_extractor is not None:
            features = self.feature_extractor(dataset["text"])
            dataset = dataset.add_column("features", [f for f in features])
                
        # performance scoring    
        if self.perf_extractor is not None:
            performances = self.perf_extractor(dataset["text"], dataset["label"])
            dataset = dataset.add_column("performance", [p for p in performances])
                
        return dataset

In [12]:
class Likelihood:
    def __init__(self):
        self.scorer = torch.nn.NLLLoss(reduction="none")
    
    def __call__(self, probs, targets, indices=None):
        return -self.scorer(probs, targets).numpy()
    
class InverseLikelihood:
    def __init__(self):
        self.scorer = torch.nn.NLLLoss(reduction="none")
    
    def __call__(self, probs, targets, indices=None):
        return 1+self.scorer(probs, targets).numpy()
    
class CleanLabSafe:
    def __init__(self):
        pass
        
    def __call__(self, probs, targets, indices=None):
        probs = probs.numpy()
        targets = targets.numpy()
        scores = ~find_label_issues(
            labels=targets,
            pred_probs=probs,
            n_jobs=1
        )
        return scores.astype(np.int32).tolist()
    
class LikelihoodShift:
    def __init__(self, original_dataset, direction="positive"):
        self.original_dataset = original_dataset
        self.direction = direction
        self.scorer = torch.nn.NLLLoss(reduction="none")
        
    def __call__(self, probs, targets, indices=None):
        new_scores  = -self.scorer(probs, targets).numpy()
        
        old_probs   = torch.tensor(self.original_dataset.select(indices)["preds"])
        old_targets = torch.tensor(self.original_dataset.select(indices)['label'])
        old_scores  = -self.scorer(old_probs, old_targets).numpy()
            
        if self.direction in "positive":
            scores = (new_scores - old_scores).clip(0, 1)
        elif self.direction in "negative":
            scores = (old_scores - new_scores).clip(0, 1)
        else:
            scores = new_scores
        return scores
        
class PerformanceExtractor:
    def __init__(self, dataset_name, scorer, model_id=None):
        self.dataset_name = dataset_name
        self.scorer = scorer
        self.model_id = model_id
        self.api = HfApi()
        self.pipe = None
        self.device = 0 if torch.cuda.is_available() else -1

        if self.model_id and not self.pipe:
            self.create_pipe(self.model_id)

        if not self.pipe:
            self.find_model_for_dataset()

    def create_pipe(self, model_id):
        self.pipe = pipeline("text-classification", 
                            model=model_id, 
                            device=self.device, 
                            padding=True, 
                            truncation=True,
                            top_k=None)
        return self.pipe

    def find_model_for_dataset(self):
        model_filter = ModelFilter(
            task="text-classification",
            library="pytorch",
            # model_name=dataset_name,
            trained_dataset=self.dataset_name)
        model_id = next(iter(self.api.list_models(filter=model_filter)))
        if model_id:
            model_id = getattr(model_id, 'modelId')
            print('Using ' + model_id + ' to support evaluation.')
            self.create_pipe(model_id)

    def extract_prediction_probabilities(self, inputs):
        output = self.pipe(inputs)
        return torch.stack([vectorize(o) for o in output])
    
    def extract_prediction_classes(self, inputs):
        return torch.argmax(self.extract_prediction_probabilities(inputs), axis=1)

    def __call__(self, inputs, targets, indices=None):
        probs   = self.extract_prediction_probabilities(inputs)
        targets = torch.tensor(targets)
        return self.scorer(probs, targets, indices)

## FADA v2.1

In [59]:
dataset_builder, dataset_config = "glue", "sst2"
task_name = "sentiment"

In [14]:
dataset = load_dataset(*dataset_config, split="train")
dataset = dataset.rename_column("sentence", "text")

2023-06-11 21:32:14,776 | datasets.builder | WARNING | Found cached dataset glue (C:/Users/Fabrice/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [15]:
nlaug_transforms = [
    # EmojifyTransformation, # UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 220: character maps to <undefined>
    SynonymInsertion,
    # BackTranslation, # runtime too long (does not attempt to use gpu) (50.15s/ba w gpu)
    # NounCompoundParaphraser, # does not return anything
    DiacriticRemoval,
    AzertyQwertyCharsSwap,
    HashtagGeneration,
    SpellingTransformation,
    TokenReplacement,
    SentenceAuxiliaryNegationRemoval,
    UseAcronyms,
    YodaPerturbation,
    # FactiveVerbTransformation, # cannot find gender.male.names
    # ProtaugmentDiverseParaphrase, # somewhat long runtime (29.63s/ba)
    # ReplaceFinancialAmount,
    AmericanizeBritishizeEnglish,
    # PunctuationWithRules,  # runtime too long (does not appear to be gpu compatible)
    # UrbanThesaurusSwap, # runtime too long (195.74s/ba)
    FillerWordAugmentation,
    SynonymSubstitution,
    # StyleTransferParaphraser, # runtime too long (192.32s/ba)
    RandomUpperPerturbation,
    WeekdayMonthAbbreviation,
    CityNamesTransformation,
    # SentenceAdditions, # runtime too long (does not attempt to use gpu) # RuntimeError: CUDA out of memory. even at batch_size=1
    ChangeDateFormat,
    # TenseTransformation, # index out of range errors / NoneType errors if not using to_tense='past'
    DyslexiaWordsSwap,
    # GenderCultureDiverseName, # does not return anything
    # GenderNeutralRewrite, # index out of range errors
    # LostInTranslation, # IndexError: index out of range in self
    AbbreviationInsertionEN,
    RandomDeletion,
    TransformerFill, # (3.20s/ba)
    ConcatMonolingual,
    CountryStateAbbreviation,
    ButterFingersPerturbation,
    # ReplaceAbbreviations, # (4.52s/ba)
    Slangificator,
    YesNoQuestionPerturbation,
    GenderSwap,
    # CloseHomophonesSwap, # runs slow, no gpu (221.04s/ba)
    SimpleCiphers,
    # ChangePersonNamedEntities, # does not return anything
    GreetingsAndFarewells,
    DiscourseMarkerSubstitution,
    # Summarization,
    VisualAttackLetters,
    # SentenceReordering, # somwhat slow (41.14s/ba)
    ChangeCharCase,
    AntonymsSubstitute,
    # MixTransliteration, # IndexError: list index out of range
    # DifferentAbilityTransformation, UnboundLocalError: local variable 'text' referenced before assignment
    Abbreviate,
    # ReplaceHypernyms, # RuntimeError: CUDA out of memory. even at batch_size=1
    # ReplaceHyponyms, # RuntimeError: CUDA out of memory. even at batch_size=1
    # PhonemeSubstitution, # IndexError: list index out of range
    # MultilingualLexiconPerturbation, FileNotFoundError: [Errno 2] No such file or directory: '/multilingual_lexicon_uncased.xz'
    # MultilingualBackTranslation, # runtime too long (does not attempt to use gpu) (98.04s/ba w gpu)
    UnitConverter,
    SentenceAdjectivesAntonymsSwitch,
    CorrectCommonMisspellings,
    # NeuralParaphaserPerturbation, # somewhat slow (4.03s/ba)
    SentenceSubjectObjectSwitch,
    # NumericToWord, # InvalidOperation: [<class 'decimal.ConversionSyntax'>]
    MultilingualDictionaryBasedCodeSwitch,
    # DiverseParaphrase, # runtime too long (does not attempt to use gpu)
    EnglishInflectionalVariation,
    ReplaceNumericalValues,
    SpeechDisfluencyPerturbation,
    WhitespacePerturbation,
    ContractionExpansions,
    ColorTransformation,
    # ContextualMeaningPerturbation, # runtime too long (not gpu compatible)
    PigLatin,
    LeetLetters,
    # HashtagifyTransformation, # error: nothing to repeat at position 0
    # CheckSrl, # RuntimeError: The size of tensor a (1127) must match the size of tensor b (512) at non-singleton dimension 1
    # GeoNamesTransformation, # does not return anything
]
nlaug_transforms = [t for t in nlaug_transforms]
nlaug_transforms = sorted(nlaug_transforms, key=lambda t: t.__name__)
nlaug_transforms = [Transform(t, task_name=task_name) for t in nlaug_transforms]

initializing Abbreviate
setting max_outputs=1
initializing AbbreviationInsertionEN
setting max_outputs=1
initializing AmericanizeBritishizeEnglish
setting max_outputs=1
initializing AntonymsSubstitute


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fabrice\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


setting max_outputs=1
initializing AzertyQwertyCharsSwap
setting max_outputs=1
initializing ButterFingersPerturbation
setting max_outputs=1
initializing ChangeCharCase
setting max_outputs=1
initializing ChangeDateFormat
setting max_outputs=1
initializing CityNamesTransformation
setting max_outputs=1
initializing ColorTransformation
setting max_outputs=1
initializing ConcatMonolingual
setting max_outputs=1
initializing ContractionExpansions
setting max_outputs=1
initializing CorrectCommonMisspellings
setting max_outputs=1
initializing CountryStateAbbreviation
setting max_outputs=1
initializing DiacriticRemoval
setting max_outputs=1
initializing DiscourseMarkerSubstitution
setting max_outputs=1
initializing DyslexiaWordsSwap
setting max_outputs=1
initializing EnglishInflectionalVariation
setting max_outputs=1
initializing FillerWordAugmentation
setting max_outputs=1
initializing GenderSwap
Loading Operation GenderSwap
setting max_outputs=1
initializing GreetingsAndFarewells
setting max_o

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fabrice\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabrice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


setting max_outputs=1
initializing SynonymSubstitution


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fabrice\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


setting max_outputs=1
initializing TokenReplacement
setting max_outputs=1
initializing TransformerFill
setting max_outputs=1
initializing UnitConverter
setting max_outputs=1
initializing UseAcronyms
setting max_outputs=1
initializing VisualAttackLetters
setting max_outputs=1
initializing WeekdayMonthAbbreviation
setting max_outputs=1
initializing WhitespacePerturbation
setting max_outputs=1
initializing YesNoQuestionPerturbation
setting max_outputs=1
initializing YodaPerturbation
setting max_outputs=1


In [16]:
blacklist = [
    sibyl.Emojify,
    sibyl.AddPositiveEmoji,
    sibyl.AddNegativeEmoji,
    sibyl.Demojify,
    sibyl.RemovePositiveEmoji,
    sibyl.RemoveNegativeEmoji,
    sibyl.AddPositiveEmoji,
    sibyl.AddNegativeEmoji,
    sibyl.InsertPositivePhrase,
    sibyl.InsertNegativePhrase,
    sibyl.AddPositiveLink,
    sibyl.AddNegativeLink,
    sibyl.ImportLinkText,
    sibyl.AddNegation,
    sibyl.RemoveNegation,
    sibyl.ChangeAntonym,
    sibyl.ConceptMix,
    sibyl.TextMix,
    sibyl.SentMix,
    sibyl.WordMix,
    sibyl.Concept2Sentence
]
sibyl_transforms = [t for t in sibyl.TRANSFORMATIONS if t not in blacklist]
sibyl_transforms = sorted(sibyl_transforms, key=lambda t: t.__name__)
sibyl_transforms = [Transform(t, task_name=task_name) for t in sibyl_transforms]

initializing AddNeutralEmoji with task_name='sentiment', return_metadata=True
initializing ChangeHypernym with task_name='sentiment', return_metadata=True
initializing ChangeHyponym with task_name='sentiment', return_metadata=True
initializing ChangeLocation with task_name='sentiment', return_metadata=True
initializing ChangeName with task_name='sentiment', return_metadata=True
initializing ChangeNumber with task_name='sentiment', return_metadata=True
initializing ChangeSynonym with task_name='sentiment', return_metadata=True
initializing ContractContractions with task_name='sentiment', return_metadata=True
initializing ExpandContractions with task_name='sentiment', return_metadata=True
initializing HomoglyphSwap with task_name='sentiment', return_metadata=True
initializing InsertPunctuationMarks with task_name='sentiment', return_metadata=True
initializing RandomCharDel with task_name='sentiment', return_metadata=True
initializing RandomCharInsert with task_name='sentiment', return_me

In [17]:
transforms = nlaug_transforms + sibyl_transforms

In [18]:
len(transforms)

66

In [19]:
def generate_array_with_one_column(N, M, column_idx):
    arr = np.zeros((N, M))
    arr[:, column_idx] = 1
    return arr

In [20]:
test_dataset = dataset.select(range(100))

In [63]:
num_examples = len(dataset)
num_transforms = len(transforms)

results = {}
for i, t in enumerate(transforms):
    
    t_name = t.transform_class.__name__
    print(t_name)
    
    policy = generate_array_with_one_column(num_examples, num_transforms, i)
    
    tracker = EmissionsTracker(project_name=f"{dataset_builder}.{dataset_config}.{t_name}.num_rows={len(test_dataset)}")

    tracker.start()
    start_time = time.time()
    augmenter = Augmenter(dataset=test_dataset, 
                          transforms=transforms, 
                          transform_probabilities=policy, 
                          num_augmentations_per_record=1,
                          num_transforms_to_apply=1,
                          batch_size=10, 
                          keep_originals=True)
    aug_dataset = augmenter.augment()
    run_time = time.time() - start_time
    emissions = tracker.stop()
    
    out = {"run_time":run_time, "applicability_rate":np.mean(aug_dataset["is_changed"]), "emissions": emissions}
    
    print(out)
    
    results[t.transform_class.__name__] = out

[codecarbon INFO @ 09:11:20] [setup] RAM Tracking...
[codecarbon INFO @ 09:11:20] [setup] GPU Tracking...
[codecarbon INFO @ 09:11:20] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:11:20] [setup] CPU Tracking...
[codecarbon WARNING @ 09:11:20] No CPU tracking mode found. Falling back on CPU constant mode.


Abbreviate


[codecarbon INFO @ 09:11:22] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:11:22] >>> Tracker's metadata:
[codecarbon INFO @ 09:11:22]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 09:11:22]   Python version: 3.8.16
[codecarbon INFO @ 09:11:22]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 09:11:22]   Available RAM : 15.912 GB
[codecarbon INFO @ 09:11:22]   CPU count: 12
[codecarbon INFO @ 09:11:22]   CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:11:22]   GPU count: 1
[codecarbon INFO @ 09:11:22]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 09:11:37] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.967082500457764 W
[codecarbon INFO @ 09:11:37] Energy consumed for all GPUs : 0.000126 kWh. Total GPU Power : 30.078 W
[codecarbon INFO @ 09:11:38] Energy consumed for all CPUs : 0.000101 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:11:38] 0.000252 kWh of electri

{'run_time': 27.932604789733887, 'applicability_rate': 0.225, 'emissions': 0.0001675710162167797}
AbbreviationInsertionEN


[codecarbon INFO @ 09:11:52] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:11:52] >>> Tracker's metadata:
[codecarbon INFO @ 09:11:52]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 09:11:52]   Python version: 3.8.16
[codecarbon INFO @ 09:11:52]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 09:11:52]   Available RAM : 15.912 GB
[codecarbon INFO @ 09:11:52]   CPU count: 12
[codecarbon INFO @ 09:11:52]   CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:11:52]   GPU count: 1
[codecarbon INFO @ 09:11:52]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 09:12:07] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.967082500457764 W
[codecarbon INFO @ 09:12:07] Energy consumed for all GPUs : 0.000128 kWh. Total GPU Power : 30.371000000000002 W
[codecarbon INFO @ 09:12:07] Energy consumed for all CPUs : 0.000097 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:12:08] 0.000250 kW

...

{'run_time': 29.64338517189026, 'applicability_rate': 0.435, 'emissions': 0.00018431961697051367}
RemoveNeutralEmoji


[codecarbon INFO @ 09:45:29] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:45:29] >>> Tracker's metadata:
[codecarbon INFO @ 09:45:29]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 09:45:29]   Python version: 3.8.16
[codecarbon INFO @ 09:45:29]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 09:45:29]   Available RAM : 15.912 GB
[codecarbon INFO @ 09:45:29]   CPU count: 12
[codecarbon INFO @ 09:45:29]   CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:45:29]   GPU count: 1
[codecarbon INFO @ 09:45:29]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 09:45:44] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.967082500457764 W
[codecarbon INFO @ 09:45:45] Energy consumed for all GPUs : 0.000130 kWh. Total GPU Power : 31.029 W
[codecarbon INFO @ 09:45:45] Energy consumed for all CPUs : 0.000101 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:45:45] 0.000256 kWh of electri

{'run_time': 29.45892643928528, 'applicability_rate': 0.0, 'emissions': 0.00017998964712496993}
WordDeletion


[codecarbon INFO @ 09:46:00] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:46:00] >>> Tracker's metadata:
[codecarbon INFO @ 09:46:00]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 09:46:00]   Python version: 3.8.16
[codecarbon INFO @ 09:46:00]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 09:46:00]   Available RAM : 15.912 GB
[codecarbon INFO @ 09:46:00]   CPU count: 12
[codecarbon INFO @ 09:46:00]   CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
[codecarbon INFO @ 09:46:00]   GPU count: 1
[codecarbon INFO @ 09:46:00]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon INFO @ 09:46:16] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.967082500457764 W
[codecarbon INFO @ 09:46:16] Energy consumed for all GPUs : 0.000129 kWh. Total GPU Power : 30.862000000000005 W
[codecarbon INFO @ 09:46:16] Energy consumed for all CPUs : 0.000097 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 09:46:16] 0.000251 kW

{'run_time': 29.548099279403687, 'applicability_rate': 0.48, 'emissions': 0.00018319329732635942}


In [64]:
augmenter_df = pd.DataFrame(results).T.reset_index().rename(columns={"index":"transform"})
augmenter_df["wasted_time"] = augmenter_df["run_time"] * (1-augmenter_df["applicability_rate"])
print(augmenter_df["run_time"].mean())
print(augmenter_df["wasted_time"].mean())
print(augmenter_df["wasted_time"].mean() / augmenter_df["run_time"].mean())

29.82666352300933
21.16594565539649
0.7096316904191561


In [67]:
emissions_df = pd.read_csv("./emissions.csv")

In [68]:
pd.concat([augmenter_df[["transform", "applicability_rate", "wasted_time"]], emissions_df], axis=1)

,transform,applicability_rate,wasted_time,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,Abbreviate,0.225,21.647769,2023-06-12T09:11:50,glue.sst2.Abbreviate.num_rows=100,113ad94f-fb89-42ac-9b2d-4bce8b083502,27.981532,0.000168,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
1,AbbreviationInsertionEN,0.225,22.348168,2023-06-12T09:12:21,glue.sst2.AbbreviationInsertionEN.num_rows=100,bb7e28fa-1140-4191-8094-8f492cd19573,28.898123,0.000179,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
2,AmericanizeBritishizeEnglish,0.500,13.457011,2023-06-12T09:12:50,glue.sst2.AmericanizeBritishizeEnglish.num_row...,ef44ca4d-7f7e-46aa-a9da-bf68202487f8,26.964852,0.000165,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
3,AntonymsSubstitute,0.320,20.989225,2023-06-12T09:13:23,glue.sst2.AntonymsSubstitute.num_rows=100,ff4200c9-c799-4f02-8694-3116dca57711,30.923853,0.000191,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
4,AzertyQwertyCharsSwap,0.045,29.782376,2023-06-12T09:13:56,glue.sst2.AzertyQwertyCharsSwap.num_rows=100,df5ffe26-119f-479c-9feb-367b3fd24038,31.218409,0.000192,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,RandomInsertion,0.480,17.111138,2023-06-12T09:44:22,glue.sst2.RandomInsertion.num_rows=100,204144aa-1f10-4b26-aedf-63f22ab21a84,32.974039,0.000205,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
62,RandomSwap,0.500,15.640733,2023-06-12T09:44:55,glue.sst2.RandomSwap.num_rows=100,256f82b4-1e7a-4adc-99b6-db9a31a10a11,31.359601,0.000195,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
63,RandomSwapQwerty,0.435,16.748513,2023-06-12T09:45:27,glue.sst2.RandomSwapQwerty.num_rows=100,12903d78-1f76-4eec-adb0-4d2d2f3ec933,29.679837,0.000184,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
64,RemoveNeutralEmoji,0.000,29.458926,2023-06-12T09:45:59,glue.sst2.RemoveNeutralEmoji.num_rows=100,682790a5-ef6d-4ff6-9f6c-20eeba6ba86b,29.520965,0.000180,0.000006,22.5,...,12,Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz,1,1 x NVIDIA GeForce RTX 2060,-97.822,37.751,15.91222,machine,N,1
